In [3]:
import numpy as np
from astropy.io import ascii, fits
from astropy.table import Table, Column
import pandas as pd

TypeError: expected string or bytes-like object

# Create a FITS for each Event

In [ ]:
hdu = fits.PrimaryHDU() #Create a HDU for extension 0

#Give it some data about the event
hdr = fits.Header()
name = 'GRB161219B-SN2016jca'
hdr['Name'] = name
hdr['Trig_t'] = '2016 December 19 18:48:39 UT'
empty_primary = fits.PrimaryHDU(header=hdr)
hdul = fits.HDUList([empty_primary])
filename = str(name+'/FITS/'+name+'.fits')
hdul.writeto(filename, overwrite=True)

# Open the new FITS file to see whats there

In [ ]:
#Open the hdul
hdul = fits.open(name+'/FITS/'+name+'.fits')
hdul[0].header

#Close it again
# hdul.close()

# XRT Data

In [ ]:
def swift_xrt(data_file, fits_file):
    table = Table.read(data_file, delimiter='\t', format='ascii', names=['time', 'dt+', 'dt-', 'flux', 'dflux+', 'dflux-'])
    table.meta['Flux'] = 'erg/cm2/s'
    table.write(fits_file, append=True)
swift_xrt(name+'/FITS/GRB161219Bxrtlc.txt', name+'/FITS/'+name+'.fits')

In [ ]:
hdul = fits.open('table.fits')
hdul[0].header

# Other Data

# Reading HTML table files

In [ ]:
file='GRB161219B-2016jca/GRB161219B-2016jca_Optical1.html'

In [ ]:
def check_float(potential_float):
    try:
        float(potential_float) #Try to convert argument into a float
        return True
    except ValueError:
        return False

n=0
k=-1
for i in range(0, 6):
    #Read in the file changing where the data starts
    data = ascii.read(file, format='html', data_start=i)
    
    
    #find the first row that has a column value that works as a float
    for j in range(len(data.columns)):
        if check_float(data[0][j])==True:
            print(i)
            print(data[0][j])
            k = i
            break
    if k==i:
        break
    n=i
    
#Get the actual data with the correct data start value    
data = ascii.read(file, format='html', data_start=k, encoding='utf-8')
data

# Cleaning up the data

* I want the pm_remover() to:

    * Take a table as input
    * Check if there are pm values in the rows
    * If there are then split that row into data and errors
    * Append the new error row to the table whilst overwriting the old row with the float data
    * Return the table

In [ ]:
# Need a way to make the columns that should be floats into floats
def pm_remover(table): #Should separate columns containing ± into data and error
    num_cols = len(table.columns) # Get the number of columns
    
    to_delete = []
    for i in range(num_cols): #Iterate over those columns
        print(num_cols)
        data_a = []
        data_b = []
        
        if isinstance(table[0][i], str)==True: #Check if theres a string in column i row 0
            for l in table.columns[i]:
                print(l)
                string = l.replace(u'\xa0', u'') #Unicode space characters are popping up when parsing the html
                
                if '–' in string:
                    sep_data_a, sep_data_b = 'NaN', 'NaN'
                    sep_data_a, sep_data_b = float(sep_data_a), float(sep_data_b) #Convert to float (nan)
                    data_a.append(sep_data_a)
                    data_b.append(sep_data_b)
                    
                if '±' in string:
                    sep_data_a, sep_data_b = l.split('±') #Separate the data
                    sep_data_a, sep_data_b = float(sep_data_a), float(sep_data_b) #Convert to float

                    data_a.append(sep_data_a)
                    data_b.append(sep_data_b)
                
            col_a = Column(data_a, name=str(table.colnames[i])+'band') #Make a new column for the data
            col_b = Column(data_b, name='Δ'+str(table.colnames[i])) #Make a new column for the errors
            
            table.add_column(col_a) #Add the data into its own column
            table.add_column(col_b) #Add the error to its own column
            
            to_delete.append(table.colnames[i])#Record that this column should be deleted.
    
    #Drop the appropriate columns one at a time
    for k in to_delete:
        print(k)
        table.remove_column(k) #Remove the columns that are being split
    
pm_remover(data)

In [ ]:
data

In [ ]:
float('NaN')

# Making FITS files from table objects

In [ ]:
data.write('GRB161219B-2016jca.fits', format='ascii', overwrite=True)

In [ ]:
data.read('GRB161219B-2016jca.fits', format='ascii')

In [ ]:
hdul = fits.open('GRB161219B-2016jca.fits')

# Template fits file